In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Rondônia - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,4.987000,0.688843,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062,13.867
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,4.987000,0.689031,2.071062e+07,1.589518e+06,11.472453,1.829323e+07,8.219943,10.777
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,4.987000,0.689218,2.073774e+07,1.591364e+06,11.476744,1.830296e+07,8.213823,13.888
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,4.987000,0.689405,2.076486e+07,1.593210e+06,11.481034,1.831268e+07,8.207703,11.859
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,4.987000,0.689592,2.079198e+07,1.595056e+06,11.485325,1.832241e+07,8.201584,14.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.931530,NaN,NaN,NaN,NaN,NaN,NaN,49.066
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.812259,NaN,NaN,NaN,NaN,NaN,NaN,49.850
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.687985,NaN,NaN,NaN,NaN,NaN,NaN,43.733
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.551024,NaN,NaN,NaN,NaN,NaN,NaN,40.078


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.115226,-1.859790,-1.684237,0.358666,-2.401614,-1.935575,1.247042
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.115226,-1.835964,-1.666360,0.383576,-2.358981,-1.906624,1.233793
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.115226,-1.812139,-1.648484,0.408485,-2.316347,-1.877673,1.220544
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.115226,-1.788313,-1.630607,0.433395,-2.273714,-1.848722,1.207295
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.115226,-1.764487,-1.612730,0.458305,-2.231081,-1.819771,1.194046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.247503,1.280168,1.167829,-1.179294,1.068920,1.031159,-1.264706
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.239967,1.265078,1.159325,-1.160375,1.051588,1.020921,-1.291394
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.218298,1.249987,1.150821,-1.141456,1.034256,1.010683,-1.318081
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.219108,1.234897,1.142317,-1.122538,1.016924,1.000446,-1.344769


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rondônia - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.115226,-1.859790,-1.684237,0.358666,-2.401614,-1.935575,1.247042
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.115226,-1.835964,-1.666360,0.383576,-2.358981,-1.906624,1.233793
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.115226,-1.812139,-1.648484,0.408485,-2.316347,-1.877673,1.220544
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.115226,-1.788313,-1.630607,0.433395,-2.273714,-1.848722,1.207295
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.115226,-1.764487,-1.612730,0.458305,-2.231081,-1.819771,1.194046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.087019,1.457304,1.215933,-1.493457,1.031936,1.145432,-0.079052
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.105967,1.457217,1.221174,-1.488264,1.051337,1.152213,-0.144286
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.124493,1.457130,1.226414,-1.483070,1.070737,1.158994,-0.209519
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.140092,1.457044,1.231655,-1.477877,1.090137,1.165775,-0.274753


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
157    29.662
158    31.408
159    29.484
160    33.601
161    36.905
Name: Rondônia - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.079360,0.470813,0.768768,-0.731913,0.443163,0.775263,-1.186007
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.020528,0.487914,0.788783,-0.758573,0.464268,0.791970,-1.119247
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.951051,0.505015,0.808798,-0.785233,0.485374,0.808677,-1.052486
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.901602,0.522117,0.828813,-0.811893,0.506479,0.825384,-0.985725
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.847437,0.539218,0.848827,-0.838553,0.527585,0.842091,-0.918965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.247503,1.280168,1.167829,-1.179294,1.068920,1.031159,-1.264706
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.239967,1.265078,1.159325,-1.160375,1.051588,1.020921,-1.291394
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.218298,1.249987,1.150821,-1.141456,1.034256,1.010683,-1.318081
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.219108,1.234897,1.142317,-1.122538,1.016924,1.000446,-1.344769


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2966103076, 1230022467, 1558704656, 1923874128, 2933099817, 575057062, 912116183, 2618872195, 599563301, 3322852787, 1463734773, 1205369941, 235879417, 2274353197, 3469219270, 462543287, 1886862105, 4214770509, 3763446533, 130201081, 3880493137, 735349186, 707964646, 458237832, 7996388, 2563993069, 1285007428, 627938948, 1631462247, 1422771919, 3426390829, 1663280473, 3192834351, 3739902212, 312228462, 3482702958, 2899397323, 1794490382, 1869144501, 2249122285, 3080586012, 2876853741, 103320711, 1961339123, 306251111, 1829348868, 2597570040, 107259297, 3081660108, 3351167184]


Step: 0 ___________________________________________
val_loss: 41.67219161987305
winner_seed: 2966103076


Step: 1 ___________________________________________
val_loss: 39.153236389160156
winner_seed: 1230022467


Step: 2 ___________________________________________
val_loss: 31.407880783081055
winner_seed: 1558704656


Step: 3 ___________________________________________
val_loss: 38.350868225097656


Step: 4 ___

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 115ms/step - loss: 2865.8420 - val_loss: 683.3258
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 498.1607 - val_loss: 739.1949
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 745.5660 - val_loss: 538.5482
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 486.0220 - val_loss: 647.9373
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 518.6790 - val_loss: 552.0594
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 489.2359 - val_loss: 526.0014
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 485.8232 - val_loss: 532.8619
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 501.3166 - val_loss: 681.2611
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 546.8483 - val_loss: 508.6959
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - lo

4/4 [==============================] - 0s 10ms/step - loss: 82.8906 - val_loss: 112.0041
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 76.0946 - val_loss: 108.5166
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 66.4413 - val_loss: 210.7669
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 118.1172 - val_loss: 101.5352
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.8889 - val_loss: 103.0555
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.6989 - val_loss: 117.1963
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 81.9114 - val_loss: 131.2479
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 73.3457 - val_loss: 98.2639
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 72.0461 - val_loss: 104.0754
Epoch 89/10000
4/4 [==============================] - 0s 8ms/step - loss: 67.0995 - va

Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 55.4967 - val_loss: 99.4720
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 66.2057 - val_loss: 104.3014
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 64.3850 - val_loss: 123.7768
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.6115 - val_loss: 118.8665
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.0805 - val_loss: 110.3054
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 68.4278 - val_loss: 107.3665
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.3177 - val_loss: 110.9502
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 65.9296 - val_loss: 116.4288
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 61.6070 - val_loss: 106.8869
Epoch 168/10000
4/4 [==============================] - 0s 9ms/ste

4/4 [==============================] - 0s 8ms/step - loss: 53.8189 - val_loss: 114.3470
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 63.0003 - val_loss: 105.1395
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 59.2186 - val_loss: 103.9555
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.7588 - val_loss: 107.2931
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 52.1484 - val_loss: 112.2813
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 71.1526 - val_loss: 105.2419
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 57.7633 - val_loss: 109.3991
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.3592 - val_loss: 107.4540
Epoch 245/10000
4/4 [==============================] - 0s 9ms/step - loss: 58.5202 - val_loss: 106.4025
Epoch 246/10000
4/4 [==============================] - 0s 10ms/step - loss: 63.4

Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 60.0394 - val_loss: 96.3784
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.6973 - val_loss: 106.0145
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.6371 - val_loss: 96.0399
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.4084 - val_loss: 90.5919
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 51.7893 - val_loss: 89.3095
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 47.8810 - val_loss: 93.8617
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 49.2609 - val_loss: 97.4264
Epoch 323/10000
4/4 [==============================] - 0s 8ms/step - loss: 56.5310 - val_loss: 98.5870
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 46.4928 - val_loss: 89.3562
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - los

...

4/4 [==============================] - 0s 8ms/step - loss: 18.0697 - val_loss: 55.5157
Epoch 1342/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.9249 - val_loss: 61.5497
Epoch 1343/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.9707 - val_loss: 49.2400
Epoch 1344/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.0196 - val_loss: 48.6130
Epoch 1345/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.4570 - val_loss: 47.4715
Epoch 1346/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.3550 - val_loss: 50.4497
Epoch 1347/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.4752 - val_loss: 53.0903
Epoch 1348/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1984 - val_loss: 59.9311
Epoch 1349/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.3405 - val_loss: 51.4910
Epoch 1350/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.24

Epoch 1420/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.5474 - val_loss: 57.4119
Epoch 1421/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.1488 - val_loss: 43.4953
Epoch 1422/10000
4/4 [==============================] - 0s 11ms/step - loss: 19.8624 - val_loss: 43.2978
Epoch 1423/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.4099 - val_loss: 44.3868
Epoch 1424/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.5061 - val_loss: 50.3168
Epoch 1425/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.4136 - val_loss: 56.3797
Epoch 1426/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.1741 - val_loss: 47.2854
Epoch 1427/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.1877 - val_loss: 57.6252
Epoch 1428/10000
4/4 [==============================] - 0s 8ms/step - loss: 19.1364 - val_loss: 47.9603
Epoch 1429/10000
4/4 [==============================] - 0s 

4/4 [==============================] - 0s 10ms/step - loss: 18.6470 - val_loss: 54.2463
Epoch 1499/10000
4/4 [==============================] - 0s 11ms/step - loss: 16.7007 - val_loss: 53.5349
Epoch 1500/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.5682 - val_loss: 51.6423
Epoch 1501/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.4605 - val_loss: 53.7616
Epoch 1502/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.8656 - val_loss: 49.3551
Epoch 1503/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.2431 - val_loss: 56.0868
Epoch 1504/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.7704 - val_loss: 49.7181
Epoch 1505/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.3864 - val_loss: 55.7542
Epoch 1506/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.5004 - val_loss: 60.0404
Epoch 1507/10000
4/4 [==============================] - 0s 9ms/step - loss: 

4/4 [==============================] - 0s 10ms/step - loss: 15.8416 - val_loss: 44.3144
Epoch 1577/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.4324 - val_loss: 41.7335
Epoch 1578/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.3725 - val_loss: 42.6774
Epoch 1579/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.1497 - val_loss: 53.9298
Epoch 1580/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.5992 - val_loss: 59.4431
Epoch 1581/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.5446 - val_loss: 53.7096
Epoch 1582/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.7173 - val_loss: 61.7527
Epoch 1583/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.4191 - val_loss: 61.2631
Epoch 1584/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.8917 - val_loss: 44.0045
Epoch 1585/10000
4/4 [==============================] - 0s 9ms/step - loss: 11

4/4 [==============================] - 0s 9ms/step - loss: 109.7261 - val_loss: 145.9608
Epoch 1655/10000
4/4 [==============================] - 0s 9ms/step - loss: 98.5598 - val_loss: 141.6013
Epoch 1656/10000
4/4 [==============================] - 0s 9ms/step - loss: 75.4914 - val_loss: 108.7755
Epoch 1657/10000
4/4 [==============================] - 0s 9ms/step - loss: 96.5498 - val_loss: 140.1461
Epoch 1658/10000
4/4 [==============================] - 0s 10ms/step - loss: 98.7531 - val_loss: 125.8889
Epoch 1659/10000
4/4 [==============================] - 0s 10ms/step - loss: 81.1607 - val_loss: 126.4470
Epoch 1660/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.2875 - val_loss: 109.2451
Epoch 1661/10000
4/4 [==============================] - 0s 11ms/step - loss: 86.3904 - val_loss: 116.5318
Epoch 1662/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.3629 - val_loss: 110.2483
Epoch 1663/10000
4/4 [==============================] - 0s 9ms/step 

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,31.628561,31.757408,31.662781,31.751755,31.722603,31.732782,31.795221,31.925695,31.891462,31.897284,31.922804,32.041637,31.742241,32.139427,32.13866,31.977001,31.905287,31.796093,32.241257,32.361774,32.344353,32.448887,33.322533,32.965382,33.989037,33.344101,34.99696,33.925022,37.393734,38.672634
Target,36.094,42.556,41.193,37.053,33.527,25.616,25.221,23.321,23.927,24.561,20.135,34.837,34.338,39.293,41.61,43.544,35.658,29.693,32.559,24.91,28.713,32.66,31.635,35.142,36.419,40.3,35.493,38.489,35.591,27.53
Error,4.465441,10.798592,9.53022,5.301247,1.804398,6.116783,6.574221,8.604696,7.964462,7.336283,11.787804,2.795364,2.59576,7.153572,9.47134,11.566998,3.752714,2.103092,0.317741,7.451775,3.631353,0.211113,1.687532,2.176617,2.429962,6.955898,0.49604,4.563976,1.802734,11.142633


In [38]:
display(mae)
display(mape)

5.419679

0.17609191

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined